In [1]:
import cv2 as cv
import numpy as np
from scipy.signal import convolve2d


def lucas_kanade(first_image_path: str, second_image_path: str):
    myframe = cv.imread(first_image_path)
    Image1 = cv.cvtColor(myframe, cv.COLOR_BGR2GRAY)

    newframe = cv.imread(second_image_path)
    Image2 = cv.cvtColor(newframe, cv.COLOR_BGR2GRAY)

    color = np.random.randint(0, 255, (100, 3))
    Gx = np.reshape(np.asarray([[-1, 1], [-1, 1]]), (2, 2))
    Gy = np.reshape(np.asarray([[-1, -1], [1, 1]]), (2, 2))
    Gt1 = np.reshape(np.asarray([[-1, -1], [-1, -1]]), (2, 2))
    Gt2 = np.reshape(np.asarray([[1, 1], [1, 1]]), (2, 2))

    Ix = (convolve2d(Image1, Gx) + convolve2d(Image2, Gx)) / 2
    Iy = (convolve2d(Image1, Gy) + convolve2d(Image2, Gy)) / 2
    It1 = convolve2d(Image1, Gt1) + convolve2d(Image2, Gt2)

    feature_params = dict(maxCorners=100,
                          qualityLevel=0.3,
                          minDistance=7,
                          blockSize=7)

    features = cv.goodFeaturesToTrack(Image1, mask=None, **feature_params)
    feature = np.int32(features)
    feature = np.reshape(feature, newshape=[-1, 2])

    u = np.ones(Ix.shape)
    v = np.ones(Ix.shape)
    status = np.zeros(feature.shape[0])
    A = np.zeros((2, 2))
    B = np.zeros((2, 1))
    mask = np.zeros_like(myframe)
    newFeature = np.zeros_like(feature)

    for a, (x, y) in enumerate(feature):

        A[0, 0] = np.sum((Ix[y - 1:y + 2, x - 1:x + 2]) ** 2)
        A[1, 1] = np.sum((Iy[y - 1:y + 2, x - 1:x + 2]) ** 2)
        A[0, 1] = np.sum(Ix[y - 1:y + 2, x - 1:x + 2] * Iy[y - 1:y + 2, x - 1:x + 2])
        A[1, 0] = np.sum(Ix[y - 1:y + 2, x - 1:x + 2] * Iy[y - 1:y + 2, x - 1:x + 2])

        B[0, 0] = -np.sum(Ix[y - 1:y + 2, x - 1:x + 2] * It1[y - 1:y + 2, x - 1:x + 2])
        B[1, 0] = -np.sum(Iy[y - 1:y + 2, x - 1:x + 2] * It1[y - 1:y + 2, x - 1:x + 2])
        prod = np.matmul(np.linalg.pinv(A), B)

        u[y, x] = prod[0]
        v[y, x] = prod[1]

        newFeature[a] = [np.int32(x + u[y, x]), np.int32(y + v[y, x])]

        if np.int32(x + u[y, x]) == x and np.int32(y + v[y, x]) == y:
            status[a] = 0
        else:
            status[a] = 1

    good_new = newFeature[status == 1]
    good_old = feature[status == 1]

    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()
        mask = cv.line(mask, (a, b), (c, d), color[i].tolist(), 2)
        newframe = cv.circle(newframe, (a, b), 5, color[i].tolist(), -1)

    return cv.add(newframe, mask)

In [2]:
result = lucas_kanade('data/LB4/frame_a.png', 'data/LB4/frame_b.png')

In [ ]:
cv.imshow('window_name', result)
cv.waitKey(0)
cv.destroyAllWindows()